In [2]:
# coding: utf-8
import pandas as pf
import os,sys,time
import shutil

class biliSpider(object):
    #创建项目
        
    def createProject(self):
        shutil.copytree('template',self.dir)  
        os.chdir(self.dir)#改变工作目录到template
        result=os.rename(u'$projectName$',self.projectName) #创建工程目录
        
        #修改scrapy.cfg 文件
        with open('scrapy.cfg','a+') as fr:
            tmp=fr.read()
        tmp=tmp.replace('$projectName$',self.projectName)
        with open('scrapy.cfg','w') as fw:
            fw.write(tmp)
        
        time.sleep(0.5)
    #配置settings
    def settings(self,mongodb_ip='192.168.11.17',mongodb_db='CrawlerData_other',mongodb_table='test'):
        #修改settings 文件
        os.chdir(self.projectName)#改变工作目录到template
        with open('settings.py','a+') as fr:
            tmp=fr.read()
        tmp=tmp.replace('$projectName$',self.projectName)
        tmp=tmp.replace('$mongodb_ip$',mongodb_ip)
        tmp=tmp.replace('$mongodb_db$',mongodb_db)
        tmp=tmp.replace('$mongodb_table$',mongodb_table)
        with open('settings.py','w') as fw:
            fw.write(tmp)
            fw.close()
    #创建items
    def createItems(self,items):
       
        items_txt=''
        for item in items:
            items_txt=items_txt+item+'=scrapy.Field()\n+    '
        with open('items.py','a+') as fr:
            tmp=fr.read()
        tmp=tmp.replace('$items$',items_txt.replace('+',''))
        with open('items.py','w') as fw:
            fw.write(tmp)
    #创建spider
    def createSpider(self):
        os.chdir(self.dir+'\\'+self.projectName+'\\'+'spiders')#改变工作目录到spiders
          #修改scrapy.cfg 文件
        with open('Spider.py','a+') as fr:
            tmp=fr.read()
        tmp=tmp.replace('$projectName$',self.projectName)
        tmp=tmp.replace('$spiderName$',self.spiderName)
        with open('Spider.py','w') as fw:
            fw.write(tmp)
        os.chdir('..\\')
        os.chdir('..\\')
        os.chdir('..\\')
    def StartReqByKw(self,url_='www.example.com/{keyword}/{page}',req_method='get',parse='self.parse_list_item'):
        os.chdir(self.dir+'\\'+self.projectName+'\\'+'spiders')#改变工作目录到spiders
        with open('Spider.py','a+') as fr:
            spiders_tmp=fr.read()
        if req_method.lower()=='get':
            req_tmp='''
    headers={
            'Accept':'application/json, text/plain, */*',
            'Content-Type':'application/x-www-form-urlencoded',
            'Host':'www.zhaodahao.com',
            'Referer':'http://www.zhaodahao.com/home/index/index.html',
            'User_Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:58.0) Gecko/20100101 Firefox/58.0',
            'X-Requested-With':'XMLHttpRequest',
        }
    def start_requests(self):
        with codecs.open('keywords.csv',encoding='utf-8') as fr:
            keywords = fr.readlines()
        page = 1
        for keyword in keywords:
            keyword=keyword.replace('\\n','')
            url_='$url_$'
            url = url_.format(keyword=keyword, page=str(page))
            item = Item()
            item['keyword'] = keyword
            item['date']=datetime.datetime.now().strftime('%Y-%m-%d')
            item['spider_name']=self.name
            req = scrapy.Request(url, callback=$parse$)
            req.meta['page'] = page
            req.meta['item'] = copy.deepcopy(item)
            req.meta['url_'] = copy.deepcopy(url_)
            yield req
        '''.replace('$url_$',url_).replace('$parse$',parse)
        else:
            req_tmp='''
        headers={
            'Accept':'application/json, text/plain, */*',
            'Content-Type':'application/x-www-form-urlencoded',
            'Host':'www.zhaodahao.com',
            'Referer':'http://www.zhaodahao.com/home/index/index.html',
            'User_Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:58.0) Gecko/20100101 Firefox/58.0',
            'X-Requested-With':'XMLHttpRequest',
        }

    def start_requests(self):
        with codecs.open('keywords.csv',encoding='utf-8') as fr:
            keywords = fr.readlines()
        page = 1
        for keyword in keywords:
            keyword=keyword.replace('\\n','')
            url_='$url_$'
            url = url_.format(keyword=keyword, page=str(page))
            
            item = Item()
            item['keyword'] = keyword
            item['date']=datetime.datetime.now().strftime('%Y-%m-%d')
            item['spider_name']=self.name
            req = scrapy.FormRequest(url,formdata= post_data,callback=$parse$)
            req.meta['page'] = page
            req.meta['item'] = copy.deepcopy(item)
            req.meta['url_'] = copy.deepcopy(url_)
            yield req
        '''.replace('$url_$',url_).replace('$parse$',parse)
                
        spiders_tmp=spiders_tmp.replace('$nextStep$',req_tmp)
        spiders_tmp=spiders_tmp.replace('$url_$',url_)
        with open('Spider.py','w') as fw:
            fw.write(spiders_tmp)
        os.chdir('..\\')
        os.chdir('..\\')
        os.chdir('..\\')

    def ParseHtml(self,res_meta_keys,line_keys,req_meta_keys,name='',next_step_type='item',callback='',method='get',TP_page_limit=10,TP_count_limit=10,TP_method='get'):
        os.chdir(self.dir+'\\'+self.projectName+'\\'+'spiders')#改变工作目录到spiders
        with open('nextStep.py','a+') as f:
            SC=f.read()
        SC=SC.replace('$name$',name)
        SC=SC.replace('$res_meta_keys$',self._create_res_meta(res_meta_keys))
        pre_res='''
        soup = BeautifulSoup(response.text)
        lines = soup.find(class_="result").find_all(class_="list-dl")
        '''
        SC=SC.replace('$pre_res$',pre_res)
        if TP_page_limit<>0 and TP_count_limit<>0:
            SC = SC.replace('$turn_page$',self._next_page(self.req_meta,method=TP_method,callback='self.'+name))
        SC=SC.replace('$line_keys$',self._create_lines_and_items(line_keys,self.items))
        SC = SC.replace('$to_next_step$', self._to_next_step(next_step_type,req_meta=req_meta_keys,method=method,callback=callback))
        with open('Spider.py','a+') as f:
            f.write(SC)
        os.chdir('..\\')
        os.chdir('..\\')
    def ParseJson(self,res_meta_keys,line_keys,req_meta_keys,name='',next_step_type='item',callback='',method='get',TP_page_limit=10,TP_count_limit=10,TP_method='get'):
        os.chdir(self.dir+'\\'+self.projectName+'\\'+'spiders')#改变工作目录到spiders
        with open('nextStep.py','a+') as f:
            SC=f.read()
        SC=SC.replace('$name$',name)
        SC=SC.replace('$res_meta_keys$',self._create_res_meta(res_meta_keys))
        pre_res='''
        json_text=response.text
        json_data=json.loads(json_text)
        lines=json_data['result']
        '''
        SC=SC.replace('$pre_res$',pre_res)
        if TP_page_limit<>0 and TP_count_limit<>0:
            SC = SC.replace('$turn_page$',self._next_page(self.req_meta,method=TP_method,callback='self.'+name))
        SC=SC.replace('$line_keys$',self._create_lines_and_items_from_json(line_keys,self.items))
        SC = SC.replace('$to_next_step$', self._to_next_step(next_step_type,req_meta=req_meta_keys,method=method,callback=callback))
        with open('Spider.py','a+') as f:
            f.write(SC)
        os.chdir('..\\')
        os.chdir('..\\')
        os.chdir('..\\')

    def _create_res_meta(self,res_meta, blank_n=8):
        script = ''
        for key in res_meta:
            script = script + key + "=response.meta['%s']\n" % key + " " * blank_n
        return script

    def _create_lines_and_items(self,lines, items, blank_n=12):
        script = ''
        for key in lines:
            script = script + key + "=line.find(class_='').find(class_='').get_text()\n" + " " * blank_n
        script = script + '\n' + " " * blank_n
        for key in items:
            if key in lines:
                script = script + "item['%s']=" % key + key + '\n' + " " * blank_n
        return script
    def _create_lines_and_items_from_json(self,lines, items, blank_n=12):
        script = ''
        for key in lines:
            script = script + key + "=line['%s']\n" % key + " " * blank_n
        script = script + '\n' + " " * blank_n
        for key in items:
            if key in lines:
                script = script + "item['%s']=" % key + key + '\n' + " " * blank_n
        return script

    def _create_req_meta_(self,req_meta, blank_n=12):
        script = ''
        for key in req_meta:
            script = script + "req.meta['{}']={}\n".format(key,key) + " " * blank_n
        return script

    def _next_page(self,req_meta, page_limit=10, count_limit=10, method='get', callback='self.parse'):

        script = '''
        if int(page) < $page$ or int(count) < $count$:
            page=str(int(page+1))
            last_url=last_url_.format(keyword=keyword, page=str(page))
            req = $method$
            req.meta['item'] = copy.deepcopy(item)
            req.meta['page'] = page
            req.meta['url_'] = copy.deepcopy(last_url_)
            $req_meta$
            yield req
    '''
        script = script.replace('$page$', str(page_limit)).replace('$count$', str(count_limit))
        if method.lower() <> 'post':
            method_sc = 'scrapy.Request(last_url, callback=$callback$)'.replace('$callback$', callback)
        else:
            method_sc = 'scrapy.FormRequest(last_url,formdata=formdata, callback=$callback$)'.replace('$callback$', callback)
        script = script.replace('$method$', method_sc)
        req_meta_sc = self._create_req_meta_(req_meta)
        script = script.replace('$req_meta$', req_meta_sc)
        return script

    def _to_next_step(self,next_step_type,req_meta=['items'], method='get', callback='self.parse'):
        # next_step_type:item or req
        if next_step_type=='req':
            script = '''
            formdata={}
            url=url_
            req = $method$
            req.meta['item'] = copy.deepcopy(item)
            req.meta['page'] = page
            req.meta['url_'] = copy.deepcopy(url_)
            $req_meta$
            yield req
    '''
            if method.lower() <> 'post':
                method_sc = 'scrapy.Request(url, callback=$callback$)'.replace('$callback$', callback)
            else:
                method_sc = 'scrapy.FormRequest(url,formdata=formdata, callback=$callback$)'.replace('$callback$', callback)
            script = script.replace('$method$', method_sc)
            req_meta_sc = self._create_req_meta_(req_meta, 12)
            script = script.replace('$req_meta$', req_meta_sc)
        else:
              script='''
            items.append(dict(item))
        req=scrapy.Request('https://www.baidu.com',callback=self.tomongodb,dont_filter=True)
        req.meta['items']=copy.deepcopy(items)
        yield req
    def tomongodb(self,response):
        items=copy.deepcopy(response.meta['items'])
        # 写入数据库，通用
        if items <> []:
            def pre_process_item(item):
                for key, value in item.iteritems():
                    if isinstance(value, str) or isinstance(value, unicode):
                        item[key] = value.replace('\\t', '').replace('\\r', '').replace('\\n', '').strip()
                return item
            items = map(pre_process_item, items)
            # 修改数据库名、数据表名
            client = MongoClient(MONGODB_IP, 27017)
            # 修改数据库名、数据表名
            db = client[MONGODB_DB]
            db_key = db[MONGODB_TABLE]
            info = db_key.insert_many(items)
              '''
        return script

    def __call__(self,dirpath,mapCrawler,spiderName,items,db_settings):
        self.dir=dirpath
        self.projectName=mapCrawler
        self.spiderName=spiderName
        self.items=items
        self.req_meta=['keyword']
        
        self.createProject()
        self.settings(db_set[0],db_set[1],db_set[2])
        self.createItems(self.items)
        os.chdir('..\\')
        os.chdir('..\\')
        self.createSpider()
        

In [6]:
os.getcwd()

'C:\\Users\\hai\\Desktop\\python\\AIscrapy\\out4'

In [103]:
os.chdir('C:\\Users\\hai\\Desktop\\python\\AIscrapy')

# 创建一个爬虫工程

In [3]:

items=['spider_name','date','app','city','brand','model','ad_content','top3_dealer','rank']
db_set=['192.168.11.17','CrawlerData_other','YADI_DB_tiba']

crawler=biliSpider()
crawler('out','AutoAd','autohome_ad',items,db_set)
crawler.StartReqByKw(req_method='get')

## 添加解析函数

In [4]:
line_keys=['spider_name','date','app','city','brand','model','ad_content','top3_dealer','rank']
res_meta_keys=['page']
req_meta_keys=[]
crawler.ParseJson(name='parse_list_item',line_keys=line_keys,res_meta_keys=res_meta_keys,next_step_type='req',TP_method='get',req_meta_keys=req_meta_keys,callback='self.parse_item',method='get')

## 添加解析函数

In [5]:
line_keys=['content']
res_meta_keys=['page']
req_meta_keys=[]
crawler.ParseHtml(name='parse_item',line_keys=line_keys,next_step_type='item',res_meta_keys=res_meta_keys,TP_method='get',req_meta_keys=req_meta_keys)